In [2]:
%reload_ext autoreload
%autoreload 2

import time
from tqdm import tqdm
import numpy as np
import scipy as sp
import torch
import torch.nn.functional as F
from torch import nn
from torchvision.transforms import v2 as transforms

from matplotlib import pyplot as plt

from networkAlignmentAnalysis.models.registry import get_model
from networkAlignmentAnalysis.datasets import get_dataset
from networkAlignmentAnalysis.experiments.registry import get_experiment
from networkAlignmentAnalysis import utils
from networkAlignmentAnalysis import files

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device: ', DEVICE)

using device:  cuda


In [8]:
# TODO
# 1.1. include additional AlignmentModel methods stored in extra class in base model

# 4. Rewrite existing analysis pipelines
# 5. SLURM!!!!

# Figure out why convolutional alignment measurement is slow...

# 2. Make CIFAR class -- prioritize --
# ----- this seems done... just need to adjust batch size and figure out normalization etcetera
# speed up loading... what the hell

# Basic alignment_comparison Analyses (or maybe for alignment_stats):
# - compare initial to final alignment...
# - compare initial alignment to delta weight norm...
# - observe alignment of delta weight
# - compare alignment to outgoing delta weight norm!

# Eigenfeature analyses:
# - Measure beta_adversarial (figure out how adversarial examples map onto eigenvectors)
# - Determine contribution of each eigenfeature on performance with a eigenvector dropout experiment
# ------ start by just looking at amplitude of activity on each eigenvector within each layer

# alignmentShaping.ipynb has an adversarial experiment worth looking at

In [83]:
model_name = 'AlexNet'
dataset_name = 'MNIST'

net = get_model(model_name, build=True, dataset=dataset_name, each_stride=True).to(DEVICE)
dataset = get_dataset(dataset_name, build=True, transform_parameters=net, device=DEVICE)

batch = next(iter(dataset.test_loader))
images, labels = dataset.unwrap_batch(batch)

# Perform forward pass
output = net(images, store_hidden=True)

# still working on if it's possible to speed up measure_alignment for convolutional layers